In [1]:
vertices = Ext, Target, Obj, Dep = ['Ext', 'Target', 'Obj', 'Dep']
links    = ((Ext, Target),
            (Target, Obj), 
            (Obj, Dep),
            (Dep, Dep))

next_state = dict(links)

from multipledispatch import dispatch
from collections      import Sequence, namedtuple
from framenet.util    import compose, groupwise, unique, juxt, aget, reduceby, iget
from pprint           import pprint

gf_tgt = compose(iget('GF', 'Target', default=None), 
                 iget(1))

def gf_or_tgt(group):
    f, s = gf_tgt(group) 
    return f or s

# Test
g1 = [(0,  {'FE': 'Agent', 'GF': 'Ext', 'PT': 'NP'}),
      (2,  {'Target': 'Target'}),
      (13, {'FE': 'Theme', 'GF': 'Obj', 'PT': 'NP'}),
      (29, {'FE': 'Path',  'GF': 'Dep', 'PT': 'PP'}),
      (50, {'FE': 'Goal',  'GF': 'Dep', 'PT': 'PP'})]    

pprint(list(map(gf_or_tgt, g1)))
    
def match(group):
    vs = map(gf_or_tgt, group)
    s  = Ext
    for v in vs:
        if v == s:
            s = next_state[s]
        else:
            return False

    return True

['Ext', 'Target', 'Obj', 'Dep', 'Dep']


In [29]:
from framenet.util import compose, juxt, ident, flatten
from functools     import reduce
from itertools     import tee

def symbol(s):
    def _symbol(xs): 
#         print('s:', s, 'xs:', xs)
        return [xs[1:]] if xs and s == xs[0] else []
    return _symbol

def seq(a, b):
    return lambda xs: reduce(union, map(b, a(xs)), [])
    
def union(xs, ys):
    return [x for x in xs if x not in ys] + ys

def alt(a, b):
    return lambda xs: union(*juxt(a, b)(xs))

epsilon = lambda xs: [xs]

def star(a):
    return alt(epsilon, lambda xs: seq(a, star(a))(xs))
    
def opt(a):
    return alt(a, epsilon)

In [30]:
ext, target, obj, dep = [symbol(s) for s in vertices]

In [31]:
ext(['Ext', 'Obj', 'Dep'])

[['Obj', 'Dep']]

In [32]:
alt(ext, target)(['Ext', 'Target'])

[['Target']]

In [33]:
alt(ext, target)(['Target', 'Ext', 'Dep'])

[['Ext', 'Dep']]

In [34]:
opt(dep)(['Dep', 'Ext', 'Obj'])

[['Ext', 'Obj'], ['Dep', 'Ext', 'Obj']]

In [35]:
seq(ext, target)(['Ext', 'Target', 'Obj'])

[['Obj']]

In [36]:
star(dep)(['Dep', 'Dep', 'Dep', 'Obj'])

[['Dep', 'Dep', 'Dep', 'Obj'], ['Dep', 'Dep', 'Obj'], ['Dep', 'Obj'], ['Obj']]